In [5]:
import numpy as np
import math
import time

#----------------------------------------------EDITABLE VARIABLES----------------------------------------------------

# Number of simulations that run in order to be averaged together.
# More runs will lead to a longer load time. 
RUNS = 110

# Number of maximum generations per simulation.
MAXGEN = 100

# This sets the number of strategies in the game.
# It should always be the same as the number of rows in the payoff matrix.
TYPES = 2

# Mutation Probability (set to zero for no mutation)
mu = 0.01

# Payoff Matrix (3 strategies)
# Symmetric games only - payoffs listed are only for player 1
matrix = np.array([[2,0],
                   [3,1]])

# Correlation parameter r - change to increase correlation or anti-correlation.
# Between 0 and 1: correlation 
# Between 0 and -1: anti-correlation
r = 0.00

#------------------------------------------------EDIT WITH CAUTION---------------------------------------------------

# Initialize the variables
pop = np.zeros(TYPES)
oldpop = np.zeros(TYPES)
stable = 0
totals = np.zeros(TYPES)
results = np.zeros((MAXGEN, TYPES))
sum_proportions = np.zeros(TYPES)

# Get a random (flat) starting distribution
def rand_fill():
    global pop
    
    # Negative log operation
    x = 0
    while x == 0:
        random = np.random.rand(TYPES)
        y = 0 - np.log(random)
        print(y)
        x = np.sum(y)
    pop = y / x
    
    print(y)

# The discrete-time replicator dynamics with mutation
def replicate():
    global pop, oldpop
    
    # Save current pop to old pop
    oldpop = np.copy(pop)
    
    # Clear payoffs
    fitness = np.zeros(TYPES)
    
    # Get new payoffs
    for i in range(TYPES):
        for j in range(TYPES):
            payoff_ij = matrix[i][j] * pop[j]
            correlated_payoff = matrix[i][j]
            fitness[i] += r * correlated_payoff + (1-r) * payoff_ij
            
            # Modify the fitness calculation to include correlation
            if i == j:
                fitness[i] += r * payoff_ij
        
    avefit = np.sum(fitness * pop)
    
    # Replicate
    for i in range(TYPES):
        pop[i] = pop[i] * ((1 - mu) * fitness[i] / avefit) + mu / TYPES


# Detect a stable state
def detect_stable(threshold=1e-5):
    global stable
    stable = np.all(np.abs(oldpop - pop) < threshold)

# Print results of run
def print_run(gen):
    global totals
    print(" ".join(map(str, pop)), " Gen: ", gen)
    totals += pop

# Prints the mean proportions across the simulations
# You can edit the number in "{:.3f}" to change the decimal point the print function truncates to
def print_results():
    mean_proportions = sum_proportions / RUNS
    formatted_proportions = ["{:.3f}".format(prop) if prop >= 0.0001 else "0.00" for prop in mean_proportions]
    
    print("\033[1m\nMean Proportions Across Simulations: \033[0m")
    print(" ".join(formatted_proportions))

# Main program
def main():
    global gen, stable, results, pop, sum_proportions
    print("\033[1mReplicator Dynamics\033[0m")
    
    # Ensure different random seeds for each run
    np.random.seed(int(time.time()))
    
    # Initialize results graph with 0's
    results = np.zeros((MAXGEN, TYPES))
    
    # Core of the program, runs for each simulation
    for i in range(RUNS):
        # Initialize variables to factory defaults
        gen = 0
        stable = 0
        rand_fill()
        
        # Runs simulation until stabilized or max generations has been reached
        while stable != 1 and gen < MAXGEN:
            replicate()
            detect_stable()
            gen += 1
            
        # Print run results, add final pop distribution to the sum array for printing full results
        print_run(gen)
        sum_proportions += pop
    print_results()
    
main()

Replicator Dynamics
[0.73818201 0.09036167]
[0.73818201 0.09036167]
0.005050349577775754 0.9949496504222242  Gen:  11
[0.18093042 0.02419995]
[0.18093042 0.02419995]
0.005050295330165031 0.994949704669835  Gen:  11
[0.96985258 0.44910563]
[0.96985258 0.44910563]
0.005050251763313009 0.994949748236687  Gen:  9
[1.73321349 0.49144592]
[1.73321349 0.49144592]
0.0050502512125282 0.9949497487874718  Gen:  10
[3.59426552 0.71104009]
[3.59426552 0.71104009]
0.005050341694922974 0.9949496583050772  Gen:  10
[0.23600204 1.06492711]
[0.23600204 1.06492711]
0.0050503348356245766 0.9949496651643754  Gen:  5
[0.07020341 0.3983336 ]
[0.07020341 0.3983336 ]
0.005050284098415164 0.9949497159015848  Gen:  5
[0.00751987 4.27738523]
[0.00751987 4.27738523]
0.005050228796854512 0.9949497712031455  Gen:  3
[0.52560907 0.18970029]
[0.52560907 0.18970029]
0.0050502839417621605 0.9949497160582379  Gen:  9
[0.68413027 0.12348047]
[0.68413027 0.12348047]
0.005050250182546263 0.9949497498174535  Gen:  11
[1.8828

In [24]:
# Correlation: 
# rπ(i,j) + (1-r)Σjπ(i,j)xj

In [25]:
# Anti-correlation: 
# Σjπ(i,j)xj + (1-r)π(i,j)